In [1]:
# Imports
# General
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier

# Keras
import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam, SGD
from keras.regularizers import L1L2, L2

# SMOTE
import imblearn
from collections import Counter
from imblearn.over_sampling import SMOTENC



# (Kernel) regularization: https://stats.stackexchange.com/questions/383310/what-is-the-difference-between-kernel-bias-and-activity-regulizers-and-when-t
# https://machinelearningmastery.com/how-to-reduce-overfitting-in-deep-learning-with-weight-regularization/

# Adaboost:
# https://stackoverflow.com/questions/39063676/how-to-boost-a-keras-based-neural-network-using-adaboost

# Xgboost:
# https://towardsdatascience.com/cage-match-xgboost-vs-keras-deep-learning-a8bb2f69a9ab

In [2]:
# Fill empty values
def fill_empty(df):
    # Get which columns have empty values
    num_nulls = df.isnull().sum()
    has_nulls = num_nulls[num_nulls > 0].to_frame()
    
    
    # Get which columns potentially are numerical based on the number of values
    num_values = df[has_nulls.index].apply(lambda x: len(x.value_counts()))
    categorical_cols = num_values[num_values<=10].to_frame().index
    
    # Get the columns with outliers
    has_outliers = []

    # For each column with nulls/nas
    for col in has_nulls.index:
        # Check if there's outliers in the column
        outliers = (np.abs(stats.zscore(df[col].dropna())) >= 3).any()
        has_outliers += [outliers]
        
    # Store which columns have outliers
    has_nulls['has_outliers'] = has_outliers  
    has_nulls = has_nulls.reset_index()
    
    nulls_and_outliers = has_nulls[has_nulls.has_outliers == True]['index']
    nulls_no_outliers = has_nulls[has_nulls.has_outliers == False]['index']

    # Fill empty values appropriately (mean for non-cat., no outliers; median for non-cat., outliers; mode for cat.)
    df[categorical_cols] = df[categorical_cols].fillna(df[categorical_cols].mode())
    df[nulls_no_outliers] = df[nulls_no_outliers].fillna(df[nulls_no_outliers].mean())
    df[nulls_and_outliers] = df[nulls_and_outliers].fillna(df[nulls_and_outliers].median())
#     df = df.fillna(0)
    return df, categorical_cols

In [3]:
# Load in the data
df = pd.read_csv('./train_data.csv')

print(df.target.value_counts())

# Make boolean columns into integer columns
df.replace({False: 0, True: 1}, inplace=True)

# Split the data into target labels y and features X
num_train = df.select_dtypes(include=np.number)
train_y = df.target
train_X = df.drop(['target'], axis = 1)

# Drop columns with too many nulls
# num_nulls = df.isnull().sum() 
# rows = train_X.shape[0]
# too_many_nulls = num_nulls[(num_nulls > rows/1.1)].to_frame()
# drop_cols = list(too_many_nulls.index)
# train_X.drop(drop_cols, axis=1, inplace=True)

# Fill empty values in the train data   
train_X, categorical_cols = fill_empty(train_X) #train_X.fillna(train_X.mean())

# Custom z-score method because scipy caused NaN problems in some columns
def z_score(arr):
    mean = np.mean(arr, axis = 0)
    std = np.std(arr, axis = 0)
    std[std == 0] = 1
    return (arr-mean)/std

col_idx = []
for col in categorical_cols:
    col_idx.append(train_X.columns.get_loc(col))

# Normalize the data using z-score standardization
train_X = z_score(train_X.to_numpy(dtype = np.float64))
train_df = pd.DataFrame(train_X)

2    1032
1     314
0     247
Name: target, dtype: int64


In [4]:
# Oversample a little bit for the target values that are underrepresented (e.g. target=0, target=1)
smote_nc = SMOTENC(categorical_features=col_idx, random_state=0, sampling_strategy = {1:450, 0:350}) # 
train_X, train_y = smote_nc.fit_resample(train_X, train_y)

In [5]:
# Turn y into categorical variable
y = to_categorical(train_y, num_classes = 3)
input_dim = train_X.shape[1]

In [6]:

train_X = pd.DataFrame(train_X)
feature_names = [f"feature {i}" for i in range(train_X.shape[1])]
forest = RandomForestClassifier(random_state=0, criterion = 'entropy')
col_importances = forest.fit(train_X, train_y).feature_importances_

columns = train_X.columns.to_frame()
columns['importance'] = col_importances
columns.sort_values(by = 'importance', ascending = False)
important_cols_names = columns[columns.importance>=columns.importance.median()]
important_cols = important_cols_names.index
# train_df = train_X[important_cols]

train_X = train_X[important_cols].to_numpy(dtype = np.float64)

In [48]:
def create_model(input_dim, dropout = 0.5, lr = 1e-2):
    initializer = keras.initializers.HeNormal()
    regularizer = L2(0.01)
    # create model
    model = Sequential()
    # Relu layers: HE15 kernel initializer
    # Maybe kernel regularization?
    model.add(Dense(256, input_dim=input_dim, activation='relu', kernel_initializer = initializer))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(Dense(64, activation='relu', kernel_initializer = initializer, kernel_regularizer = regularizer)) 
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(Dense(64, activation='relu', kernel_initializer = initializer, kernel_regularizer = regularizer))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(Dense(3, activation='softmax'))
    # keras.metrics.Accuracy(), keras.metrics.Precision(), keras.metrics.Recall()
    model.compile(loss='categorical_crossentropy', optimizer=SGD(learning_rate=lr, momentum = 0.9, nesterov=True), metrics=['accuracy'])
    return model

def avg(arr):
    return sum(arr)/len(arr)

# kfold = KFold(n_splits=10, shuffle=True)

In [46]:
from sklearn.ensemble import AdaBoostClassifier
# class CategoricalClassifier():
#     def __init__(self):
#         super().__init__()

#     def fit(self, X, y, batch_size = 32, n_epochs = 40):
#         self.classifier = create_model(X.shape[1])

#         self.classifier.fit(X, y, batch_size = batch_size, epochs = n_epochs, verbose = 0)
#         return self

#     def transform(self, X, y=None):
#         return np.argmax(self.classifier.predict(X), axis = -1)

#     def predict(self, X, y=None):
#         return self.transform(X)
# Make a train-validation split
# X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.1, shuffle = True)

# estimator = KerasClassifier(build_fn=create_model, epochs=40, batch_size=32, verbose=1)
# boosted_ann = AdaBoostClassifier(base_estimator= estimator)
# boosted_ann.fit(X_train, y_train)# scale your training data 
# boosted_ann.predict(rescaledX_Test)

In [ ]:
#https://stackoverflow.com/questions/47892505/dropout-rate-guidance-for-hidden-layers-in-a-convolution-neural-network
def train_network(model, X_train, y_train, X_val, y_val, 
                  n_epochs = 10, batch_size = 32, verbose = 1, save = False, path = 'best_model'):

    train_losses = []
    train_accs = []
    val_losses = []
    val_accs = []

    best_val_acc = 0
    best_val_loss = 1
    best_epoch = 0

    # Loop over epochs
    for epoch in range(n_epochs):

        # Train model
        results = model.fit(X_train, y_train, batch_size = batch_size, verbose = verbose)

        # Get training loss and accuracy and append to lists
        train_losses.append(results.history['loss'])
        train_accs.append(results.history['accuracy'])

        # Get performance (i.e. loss and accuracy) on validation set
        val_loss, val_acc = model.evaluate(X_val, y_val, verbose = verbose)

        # Append to lists
        val_losses.append(val_loss)
        val_accs.append(val_acc)

        # Keep track of which epoch had the best validation accuracy and store the corresponding model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_epoch = epoch
#             if save:
#                 model.save(path)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            if save:
                model.save(path)

    # Visualise the performance of the model for the epochs
    if verbose > 0:
        fig = plt.figure(figsize=(8, 4))
        train_loss_plt = plt.plot(train_losses) 
        train_accs_plt = plt.plot(train_accs)
        val_loss_plt = plt.plot(val_losses)
        val_acc_plt = plt.plot(val_accs)

        # Text and legend for the plot
        plt.legend(['train loss', 'train accuracy', 'validation loss', 'validation accuracy'],
                  loc='upper left', bbox_to_anchor=(1, 1))
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.title('Model performance')

        plt.show()

        # Print the performance values (accuracies, loss) for train, validation
        print('Train accuracy:\t\t\t{:.2f}\tTrain loss: {:.2f}'.format(train_accs[-1][0], train_losses[-1][0]))
        print('Validation accuracy:\t\t{:.2f}\tValidation loss: {:.2f}'.format(val_accs[-1], val_losses[-1]))
        print('Best validation accuracy:\t{:.2f} (epoch {:})'.format(best_val_acc, best_epoch))

# Make a train-validation split
X_train, X_val, y_train, y_val = train_test_split(train_X, y, test_size=0.1, shuffle = True)
model = create_model(train_X.shape[1])
train_network(model, X_train, y_train, X_val, y_val, n_epochs = 30, save = True, batch_size = 16)

6/6 [==============================] - 0s 1ms/step - loss: 0.9988 - accuracy: 0.7065
INFO:tensorflow:Assets written to: best_model\assets
6/6 [==============================] - 0s 1ms/step - loss: 0.9083 - accuracy: 0.7500
INFO:tensorflow:Assets written to: best_model\assets
6/6 [==============================] - 0s 1ms/step - loss: 0.8946 - accuracy: 0.7337
INFO:tensorflow:Assets written to: best_model\assets
6/6 [==============================] - 0s 1ms/step - loss: 0.8424 - accuracy: 0.7391
INFO:tensorflow:Assets written to: best_model\assets
6/6 [==============================] - 0s 999us/step - loss: 0.8210 - accuracy: 0.7500
INFO:tensorflow:Assets written to: best_model\assets
6/6 [==============================] - 0s 1ms/step - loss: 0.7844 - accuracy: 0.7663
INFO:tensorflow:Assets written to: best_model\assets
6/6 [==============================] - 0s 1ms/step - loss: 0.7423 - accuracy: 0.7772
INFO:tensorflow:Assets written to: best_model\assets
6/6 [===========================

In [51]:
# Create prediction dataset
from tensorflow import keras
model = keras.models.load_model('best_model')
naming_cols = ['opportunity_id', 'current_date_day', 'current_date_month', 'current_date_year']
test_df = pd.read_csv('./data2tal_testupload.csv')
index_names = pd.read_csv('./check_upload_kaggle.csv')
# test_df.drop(drop_cols, axis=1, inplace=True)
test_df, cat_ = fill_empty(test_df)

test_df.columns = list(range(test_df.shape[1]))
test_df = test_df[important_cols]
train_X_df = pd.DataFrame(train_X)
train_X_df.columns = list(range(train_X_df.shape[1]))
test_df.columns = list(range(train_X_df.shape[1]))
test_df = test_df.fillna(-1) #train_X_df.mean()
test_df = test_df.to_numpy(dtype = np.float64)
test_X = z_score(test_df)
pred = model.predict(test_X)
print(pred)
pred_target = np.argmax(pred, axis = 1)
print(pred_target)
df_pred = pd.DataFrame([])
df_pred['index'] = index_names['index']
df_pred['target'] = pred_target

df_pred.to_csv('./prediction.csv', index = False)

[[1.45600911e-03 3.62062268e-03 9.94923413e-01]
 [2.12711887e-03 5.90081699e-03 9.91972089e-01]
 [9.07395128e-03 2.20946278e-02 9.68831480e-01]
 [9.34596211e-02 2.66224474e-01 6.40315890e-01]
 [2.50455160e-02 3.34315747e-02 9.41522956e-01]
 [2.54195798e-02 3.11638452e-02 9.43416536e-01]
 [1.21988937e-01 1.96874678e-01 6.81136310e-01]
 [2.66686976e-02 2.71334618e-01 7.01996684e-01]
 [2.05547418e-02 7.61334538e-01 2.18110666e-01]
 [1.99767854e-02 5.12814820e-01 4.67208356e-01]
 [7.14913756e-02 2.15444863e-01 7.13063717e-01]
 [1.76199839e-01 3.99056584e-01 4.24743593e-01]
 [9.99036729e-02 2.49544352e-01 6.50552034e-01]
 [1.10010982e-01 3.61825794e-01 5.28163195e-01]
 [2.14056104e-01 1.91294849e-01 5.94649017e-01]
 [1.11414514e-01 4.87378925e-01 4.01206642e-01]
 [5.04498668e-02 9.95576605e-02 8.49992454e-01]
 [1.25700003e-02 1.55789226e-01 8.31640840e-01]
 [4.53325082e-03 2.97957212e-02 9.65671003e-01]
 [1.68079183e-01 6.53016686e-01 1.78904101e-01]
 [3.39835376e-01 3.10455829e-01 3.497088

In [11]:
def get_rows_with_nulls(df):
    num_nulls = df.isna().sum()
    return num_nulls[num_nulls > 0]

test_data = pd.read_csv('./data2tal_testupload.csv')
train_data = pd.read_csv('./train_data.csv')

test_nulls = get_rows_with_nulls(test_data)
test_rows = test_data.shape[0]
fully_empty_test = list(test_nulls[test_nulls >= test_rows].index)

train_nulls = get_rows_with_nulls(train_data)
train_rows = train_data.shape[0]
mostly_empty_train = list(train_nulls[train_nulls>= train_rows/1.15].index)

def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

intersect = intersection(mostly_empty_train, fully_empty_test)
print(len(intersect))

496


In [12]:
# 506     188 (filled)
# 896     188 (filled)
# 1754    188
# 1904    188
# 2202    188
# 2392    188

In [13]:
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer


# df = pd.read_csv('./train_data.csv')

# print(df.target.value_counts())

# # Make boolean columns into integer columns
# df.replace({False: 0, True: 1}, inplace=True)

# # Split the data into target labels y and features X
# num_train = df.select_dtypes(include=np.number)
# # train_y = df.target
# # train_X = df.drop(['target'], axis = 1)
# np_df = df.to_numpy()

# imp = IterativeImputer(max_iter=10, random_state=0)
# imp.fit(np_df)

# # the model learns that the second feature is double the first
# print(np.round(imp.transform(np_df)))
